# Task 3: Modeling Optimization

**Course:** Introduction to Data Science
**Lecturer:** Prof. Dr. Hendrik Meth

**Group 2:**
- Linus Breitenberger
- Tristan Ruhm
- Prarichut Poachanuan
- Anushka Irphale
- Patryk Gadziosmki

<div style="width:100%;height:30px;background-color:#E31134"></div>

<div style="width:100%;height:30px;background-color:#E31134"></div>

## 6. Summary of Task 3

# 1. Reworking and Optimization of train and test data

Since our final model for Task 2 didn't seem to be optimal and some details had been overseen, we decided to do a rework of our train and test datas and optimize it.

Because of the high correlation with 'cnt', we decided to keep 'instant' in the data set. The features 'temp' and 'atemp' had a multicorrelation, so we kept 'atemp', which had a minimal better correlation, in and dropped 'temp'. We also removed the 'casual' and 'registered' labels, since we only use 'cnt'.

In the end we tested our models with the new test data and had following result:
- MAE: 923.977 
- R^2 value of the model:  0.36870153666640637

The Mean Absolute Error has an improvement. On the opposite, the determination R^2 has a deterioration. We exported the reworked train- and testdata to csv and used for the model optimization.


# 2. Model Optimization